In [1]:
print("hello world")

hello world


In [2]:
import os

In [4]:
os.chdir("d:/WorkANDProjects/Kidney-Disease-Classification-Deep-Learning-Project")

In [5]:
%pwd

'd:\\WorkANDProjects\\Kidney-Disease-Classification-Deep-Learning-Project'

In [7]:
pip install dagshub

   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/14.0 MB 10.1 MB/s eta 0:00:02
   ----------- ---------------------------- 3.9/14.0 MB 10.2 MB/s eta 0:00:01
   ----------------- ---------------------- 6.3/14.0 MB 10.2 MB/s eta 0:00:01
   ------------------------ --------------- 8.7/14.0 MB 10.5 MB/s eta 0:00:01
   -------------------------------- ------- 11.3/14.0 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------  13.9/14.0 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 14.0/14.0 MB 10.7 MB/s  0:00:01
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----------------------- ---------------- 2.4/4.0 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 11.0 MB/s  0:00:00

   -------- -------------------------------  4/19 [marshmallow]
   ---------- -----------------------------  5/19 [lxml]
   ------------ --------------------------

In [9]:
import dagshub
dagshub.init(repo_owner='MunimAkbar', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2a6709b5-be2d-4dd3-9d65-531b7f839112&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ff50b29e608d03476893cf7d9d1b08bb6111e32a9c3dae05bf1e6ad663dfd246




Accessing as MunimAkbar

Initialized MLflow to track repo "MunimAkbar/Kidney-Disease-Classification-Deep-Learning-Project"

Repository MunimAkbar/Kidney-Disease-Classification-Deep-Learning-Project initialized!

c:\Users\pc\anaconda3\envs\kidney\Lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [1]:
import dagshub
dagshub.init(repo_owner='MunimAkbar', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as MunimAkbar

Initialized MLflow to track repo "MunimAkbar/Kidney-Disease-Classification-Deep-Learning-Project"

Repository MunimAkbar/Kidney-Disease-Classification-Deep-Learning-Project initialized!

c:\Users\pc\anaconda3\envs\kidney\Lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [11]:

import tensorflow as tf

In [12]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [13]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [14]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/MunimAkbar/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [16]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [19]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-30 21:02:19,163: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-30 21:02:19,174: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-30 21:02:19,176: INFO: common: created directory at: artifacts]
Found 300 images belonging to 2 classes.
10/10 [==============================] - 28s 3s/step - loss: 15.6398 - accuracy: 0.5000
[2025-08-30 21:02:47,722: INFO: common: json file saved at: scores.json]


2025/08/30 21:02:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-30 21:02:50,202: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\pc\AppData\Local\Temp\tmpl3quf3nz\model\data\model\assets
[2025-08-30 21:02:50,770: INFO: builder_impl: Assets written to: C:\Users\pc\AppData\Local\Temp\tmpl3quf3nz\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/08/30 21:03:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
